79. Word Search
127. Word Ladder
126. Word Ladder II
130. Surrounded Regions，被X包围的O改变成X，从边界出发，对边界能蔓延到的O不作处理，其余的全部更改
199. Binary Tree Right Side View
513. Find Bottom Left Tree Value
515. Find Largest Value in Each Tree Row
    [leaf for node in queue for leaf in (node.left,node.right) if leaf]
    利用深度参数作DFS https://leetcode.com/problems/find-largest-value-in-each-tree-row/discuss/99018/Python-BFS-and-DFS
200. Number of Islands，给定矩形往下遍历
207. course schedule
416. Partition Equal Subset Sum，将列表平分为两等分
417. Pacific Atlantic Water Flow
490. The Maze
529. Minesweeper，给定矩形往下遍历
542. 01 Matrix，给定矩形往下遍历
695. Max Area of Island
787. Cheapest Flights Within K Stops

In [7]:
import collections

In [2]:
# 200. Number of Islands
# BFS和DFS界限并不明显
# DFS: 找到一个满足的，用递归一直往下走，走的过程中完成操作
# BFS：找到一个满足的，将下一步都存入队列，左边pop出
def numIslands(grid):
    """
    :type grid: List[List[str]]
    :rtype: int
    """
    if not grid: return 0
    r, c = len(grid), len(grid[0])
    visited = [[False for _ in range(c)] for _ in range(r)]

    def dfs(i, j):
        if i < 0 or i >= r or j < 0 or j >= c or grid[i][j] == '0' or visited[i][j]:
            return
        visited[i][j] = True
        dfs(i + 1, j)
        dfs(i - 1, j)
        dfs(i, j + 1)
        dfs(i, j - 1)

    count = 0
    for i in range(r):
        for j in range(c):
            if not visited[i][j] and grid[i][j] == '1':
                dfs(i, j)
                count += 1
    return count

numIslands([["1","1","1","1","0"],["1","1","0","1","0"],["1","1","0","0","0"],["0","0","0","0","0"]])

# 694. Number of Distinct Islands，考虑形状
import collections
def numDistinctIslands(grid):
    """
    :type grid: List[List[int]]
    :rtype: int
    """
    m,n=len(grid),len(grid[0])
    directions = [(-1,0),(1,0),(0,-1),(0,1)]
    visited = [[False for _ in range(n)] for _ in range(m)]
    islands = dict()#用一个字典来统计形状，value为相对值

    def dfs(i,j):
        visited[i][j]=True
        queue = collections.deque([(i,j)])
        curisland = {(i,j):set()}
        while queue:
            ii,jj = queue.popleft()
            for dx,dy in directions:
                x = ii+dx
                y = jj+dy
                if 0<=x<m and 0<=y<n and grid[x][y]==1 and visited[x][y] is False:
                    visited[x][y] = True
                    queue.append((x,y))
                    curisland[(i,j)].add((x-i,y-j))
        return isDistinct(curisland,i,j)

    def isDistinct(curisland,i,j):
        for island in islands:
            if len(islands[island]) == len(curisland[(i,j)]) and islands[island] == curisland[(i,j)]:
                return 0
        islands[(i,j)] = curisland[(i,j)]
        return 1

    count = 0
    for i in range(m):
        for j in range(n):
            if grid[i][j] and visited[i][j] is False:
                count+=dfs(i,j)
    return count
numDistinctIslands([[1,1,0,1,1],[1,0,0,0,0],[0,0,0,0,1],[1,1,0,1,1]])

# 711. Number of Distinct Islands II
def numDistinctIslands2(grid):
    """
    :type grid: List[List[int]]
    :rtype: int
    """
    m,n = len(grid),len(grid[0])
    directions = [(-1,0),(1,0),(0,-1),(0,1)]
    islands = set()
    visited = [[False for _ in range(n)] for _ in range(m)]

    def dfs(i,j):
        visited[i][j] = True
        stack = [(i,j)]
        island = [(i,j)]
        while stack:
            i,j = stack.pop()
            for dx,dy in directions:
                x,y = i+dx,j+dy
                if 0<=x<m and 0<=y<n and not visited[x][y] and grid[x][y]:
                    island.append((x,y))
                    stack.append((x,y))
                    visited[x][y] = True
        return isDistinct(island)
        
    def isDistinct(island):
        island = sorted(island,key=lambda x:(x[0],x[1]))
        island0 = getShape(island)
        if island0 in islands:
            return 0
        islands.add(island0)

        islands.add(getShape(sorted([(i,-j) for i,j in island], key=lambda x:(x[0],x[1]))))
        islands.add(getShape(sorted([(-i,-j) for i,j in island], key=lambda x:(x[0],x[1]))))
        islands.add(getShape(sorted([(-i,j) for i,j in island], key=lambda x:(x[0],x[1]))))
        islands.add(getShape(sorted([(j,i) for i,j in island], key=lambda x:(x[0],x[1]))))
        islands.add(getShape(sorted([(j,-i) for i,j in island], key=lambda x:(x[0],x[1]))))            
        islands.add(getShape(sorted([(-j,i) for i,j in island], key=lambda x:(x[0],x[1]))))
        islands.add(getShape(sorted([(-j,-i) for i,j in island], key=lambda x:(x[0],x[1]))))

        return 1

    def getShape(island):
        return tuple((i-island[0][0],j-island[0][1]) for i,j in island)

    res = 0
    for i in range(m):
        for j in range(n):
            if not visited[i][j] and grid[i][j]:
                res+=dfs(i,j)

    return res               

# 827. Making A Large Island，可以将其中的一个0换成1，求最大的岛屿
# 先将所有的岛屿面积处理出来，然后对每个0的四周求和
def largestIsland(grid):
    """
    :type grid: List[List[int]]
    :rtype: int
    """
    if not grid or not grid[0]:
        return 0
    m,n = len(grid),len(grid[0])
    visited = [[False for _ in range(n)] for _ in range(m)]
    counts = dict()
    cur = 1

    def dfs(i,j,cur):
        level = [(i,j)]
        visited[i][j]=True
        area = 0
        while level:
            i,j = level.pop()
            grid[i][j] = cur
            area+=1
            for dx,dy in [(-1,0),(1,0),(0,1),(0,-1)]:
                x,y = i+dx,j+dy
                if 0<=x<m and 0<=y<n and not visited[x][y] and grid[x][y]:
                    visited[x][y]=True
                    level.append((x,y))
        counts[cur] = area
        cur+=1
        return cur

    for i in range(m):
        for j in range(n):
            if grid[i][j] and not visited[i][j]:
                cur = dfs(i,j,cur)

    maximum = 1
    for i in range(m):
        for j in range(n):
            if grid[i][j]==0:
                temp = set()
                for dx,dy in [(-1,0),(1,0),(0,1),(0,-1)]:
                    x,y = i+dx,j+dy
                    if 0<=x<m and 0<=y<n and grid[x][y]:
                        temp.add(grid[x][y])
                maximum = max(maximum,sum(counts[t] for t in temp)+1)
    if not counts:
        return 1
    return max(maximum,max(counts.values()))

largestIsland([[1, 1], [1, 0]])

4

In [12]:
# 959. Regions Cut By Slashes
def regionsBySlashes(grid):
    """
    :type grid: List[str]
    :rtype: int
    """
    n = len(grid)
    arr = [[0 for _ in range(3*n)] for _ in range(3*n)]

    for i in range(n):
        for j in range(n):
            if grid[i][j]==" ":
                continue
            elif grid[i][j] == "/":
                arr[3*i][3*j+2]=1
                arr[3*i+1][3*j+1]=1
                arr[3*i+2][3*j] = 1
            else:
                arr[3*i][3*j],arr[3*i+1][3*j+1],arr[3*i+2][3*j+2]=[1,1,1]

    visited = [[False for _ in range(3*n)] for _ in range(3*n)]

    def dfs(i,j):
        for dx,dy in [(-1,0),(1,0),(0,1),(0,-1)]:
            x,y = i+dx,j+dy
            if 0<=x<3*n and 0<=y<3*n and not visited[x][y] and arr[i][j]==0:
                visited[x][y] = True
                dfs(x,y)

    res = 0
    for i in range(3*n):
        for j in range(3*n):
            if not visited[i][j] and arr[i][j]==0:
                visited[i][j] = True
                res+=1
                dfs(i,j)
    return res
regionsBySlashes(["//","/ "])

3

In [2]:
# 947. Most Stones Removed with Same Row or Column，同行或者同列可删除，最多多少个
# 如果两个石头在同行或者同列，两个石头就是连接的。连在一起的石头，可以组成一个连通图。每一个连通图至少会剩下1个石头。
import collections
def removeStones(stones):
    """
    :type stones: List[List[int]]
    :rtype: int
    """
    rows = collections.defaultdict(set)
    cols = collections.defaultdict(set)
    for i, j in stones:
        rows[i].add(j)#第i行有哪些元素
        cols[j].add(i)#第j行有哪些元素

    def dfsRow(i):
        seenR.add(i)
        for j in rows[i]:
            if j not in seenC:
                dfsCol(j)

    def dfsCol(j):
        seenC.add(j)
        for i in cols[j]:
            if i not in seenR:
                dfsRow(i)

    seenR, seenC = set(), set()
    islands = 0
    for i, j in stones:
        if i not in seenR:
            islands += 1
            dfsRow(i)
            dfsCol(j)
    return len(stones) - islands


def removeStones(stones):
    UF = {}
    def find(x):
        if x != UF[x]:
            UF[x] = find(UF[x])
        return UF[x]
    def union(x, y):
        UF.setdefault(x, x)
        UF.setdefault(y, y)
        UF[find(x)] = find(y)

    for i, j in stones:
        union(i, ~j)
    return len(stones) - len({find(x) for x in UF})
removeStones(stones = [[0,0],[0,1],[1,0],[1,2],[2,1],[2,2]])

5

In [ ]:
# 542. 01 Matrix
def updateMatrix(matrix):
    q, m, n = [], len(matrix), len(matrix[0])
    for i in range(m):
        for j in range(n):
            if matrix[i][j] != 0:
                matrix[i][j] = 0x7fffffff
            else:
                q.append((i, j))
    for i, j in q:
        for r, c in ((i, 1+j), (i, j-1), (i+1, j), (i-1, j)):
            z = matrix[i][j] + 1
            if 0 <= r < m and 0 <= c < n and matrix[r][c] > z:
                matrix[r][c] = z
                q.append((r, c))
    return matrix

In [3]:
# 199. Binary Tree Right Side View
#保存树每层最右一个元素
# BFS: 层序遍历，用栈提取最后一个元素
# DFS：从右到左深度优先，如果深度达到预定长度时有值，则保留

In [30]:
# 207. course schedule
def canFinish(numCourses, prerequisites):
    """
    :type numCourses: int
    :type prerequisites: List[List[int]]
    :rtype: bool
    """
    courses = {i for i in range(numCourses)}
    while True:
        if len(courses) == 0:
            return True
        cur = set()
        for i in prerequisites:
            cur.add(i[0])
        completed,courses = courses - cur,cur
        if len(completed)==0 and len(courses)!=0:
            return False
        prerequisites = [i for i in prerequisites if i[1] not in completed]
# canFinish(6, [[1,3],[0,3],[3,4],[3,2],[4,2]] )
canFinish(5,[[0,1],[2,1],[3,0],[2,0],[4,1]])

True

In [67]:
# BFS,从后向前
def canFinish1(numCourses, prerequisites):
    forward = {i: set() for i in range(numCourses)}
    backward = collections.defaultdict(set)
    for i, j in prerequisites:
        forward[i].add(j)
        backward[j].add(i)
    print('forward',forward)
    print('backward',backward)
    queue = collections.deque([node for node in forward if len(forward[node]) == 0])
    print(queue)
    while queue:
        node = queue.popleft()
        for neigh in backward[node]:
            forward[neigh].remove(node)
            if len(forward[neigh]) == 0:
                queue.append(neigh)
        forward.pop(node)
    return not forward  # if there is cycle, forward won't be None
# BFS,从前向后
def canFinish2(numCourses, prerequisites):
    forward = {i: set() for i in range(numCourses)}
    backward = collections.defaultdict(set)
    for i, j in prerequisites:
        forward[i].add(j)
        backward[j].add(i)
    queue = collections.deque([node for node in range(numCourses) if not backward[node]])
    count = 0
    print('forward',forward)
    print('backward',backward)
    print(queue)
    while queue:
        node = queue.popleft()
        count += 1
        for neigh in forward[node]:# 对于有该先修课程的课程，去掉该边，如果这个节点无入度，则加入列表
            backward[neigh].remove(node)
            if not backward[neigh]:
                queue.append(neigh)
    return count == numCourses
# DFS
def canFinish(numCourses, prerequisites):
    graph = [[] for _ in range(numCourses)]# 每个顶点的下家
    visit = [0 for _ in range(numCourses)]
    for x, y in prerequisites:
        graph[x].append(y)
    print(graph,visit)
    def dfs(i):
        if visit[i] == -1:#如果正在被访问，则返回False
            return False
        if visit[i] == 1:#如果被访问过，则返回True
            return True
        visit[i] = -1#正在被访问
        for j in graph[i]:
            if not dfs(j):
                return False
        visit[i] = 1
        return True
    for i in range(numCourses):
        if not dfs(i):
            return False
    return True
canFinish2(5, [[1,3],[0,3],[3,4],[3,2],[4,2]] )

forward {0: {3}, 1: {3}, 2: set(), 3: {2, 4}, 4: {2}}
backward defaultdict(<class 'set'>, {3: {0, 1}, 4: {3}, 2: {3, 4}, 0: set(), 1: set()})
deque([0, 1])


True

In [105]:
# 上述拓扑排序的顺序，好像我弄反了
def findOrder(numCourses, prerequisites):
    forward = {i: set() for i in range(numCourses)}
    backward = {i: set() for i in range(numCourses)}
    for i, j in prerequisites:
        forward[i].add(j)
        backward[j].add(i)
    queue = [i for i in backward if len(backward[i])==0]
    res = []
    counts = 0
    print('forward',forward)
    print('backward',backward)
    while queue:
        node = queue.pop(0)
        res.append(node)
        for i in forward[node]:
            backward[i].remove(node)
            if len(backward[i])==0:
                queue.append(i)
        counts+=1
    if counts!=numCourses:
        return []
    return res[::-1]

In [106]:
findOrder(6,[[3,1],[3,0],[2,3],[4,3],[2,4],[1,5],[0,5]])

forward {0: {5}, 1: {5}, 2: {3, 4}, 3: {0, 1}, 4: {3}, 5: set()}
backward {0: {3}, 1: {3}, 2: set(), 3: {2, 4}, 4: {2}, 5: {0, 1}}


[5, 1, 0, 3, 4, 2]

In [157]:
# 329. Longest Increasing Path in a Matrix
# 关于需要类似迭代求解的，用DFS，用visit矩阵记录是否遍历过，思考DFS的过程中是否会找到终点
def longestIncreasingPath(matrix):
    if not matrix or not matrix[0]: return 0
    m,n = len(matrix),len(matrix[0])
    path = [[0 for i in range(n)] for _ in range(m)]
    
    def dfs(i,j):
        if path[i][j]:
            return path[i][j]
        return max([dfs(i-1,j) if i>=1 and matrix[i][j]>matrix[i-1][j] else 0\
                   ,dfs(i+1,j) if i<m-1 and matrix[i][j]>matrix[i+1][j] else 0\
                   ,dfs(i,j-1) if j>=1 and matrix[i][j]>matrix[i][j-1] else 0\
                   ,dfs(i,j+1) if j<n-1 and matrix[i][j]>matrix[i][j+1] else 0])+1
    
    for i in range(m):
        for j in range(n):
            path[i][j]=dfs(i,j)
    return max([max(i) for i in path])
matrix=[[0,1,2,3,4,5,6,7,8,9],[19,18,17,16,15,14,13,12,11,10],[20,21,22,23,24,25,26,27,28,29],[39,38,37,36,35,34,33,32,31,30],[40,41,42,43,44,45,46,47,48,49],[59,58,57,56,55,54,53,52,51,50],[60,61,62,63,64,65,66,67,68,69],[79,78,77,76,75,74,73,72,71,70],[80,81,82,83,84,85,86,87,88,89],[99,98,97,96,95,94,93,92,91,90],[100,101,102,103,104,105,106,107,108,109],[119,118,117,116,115,114,113,112,111,110],[120,121,122,123,124,125,126,127,128,129],[139,138,137,136,135,134,133,132,131,130],[0,0,0,0,0,0,0,0,0,0]]
longestIncreasingPath(matrix)

140

In [184]:
# 529. Minesweeper
# 如果访问过，就不管；统计周围雷的个数，如果非0，则记为M，如果为0，则记为B，并且访问周围元素
# 将要处理的节点存入栈后，DFS从后向前处理，BFS从前向后处理
# https://leetcode.com/problems/minesweeper/discuss/144746/Python-BFS-%2B-DFS-with-comments
def updateBoard(board, click):
    """
    :type board: List[List[str]]
    :type click: List[int]
    :rtype: List[List[str]]
    """
    m,n=len(board),len(board[0])
    visited = [[False for _ in range(n)] for _ in range(m)]

    def update(i,j):
        if visited[i][j]:
            return board[i][j]
        if board[i][j]=='M':
            visited[i][j]=True
            return 'X'
        count = 0
        for point in ([i-1,j-1],[i-1,j],[i-1,j+1],[i,j-1],[i,j],[i,j+1],[i+1,j-1],[i+1,j],[i+1,j+1]):
            x,y = point[0],point[1]
            if 0<=x<=m-1 and 0<=y<=n-1:
                if board[x][y]=='M':
                    count+=1
        if count!=0:
            visited[i][j]=True
            return count
        else:
            visited[i][j]=True
            for point in ([i-1,j-1],[i-1,j],[i-1,j+1],[i,j-1],[i,j],[i,j+1],[i+1,j-1],[i+1,j],[i+1,j+1]):
                x,y = point[0],point[1]
                if 0<=x<=m-1 and 0<=y<=n-1:
                    board[x][y]=update(x,y)
            return 'B'

    board[click[0]][click[1]] = update(click[0],click[1])
    return board

board = [['E', 'E', 'E', 'E', 'E'],
 ['E', 'E', 'M', 'E', 'E'],
 ['E', 'E', 'M', 'E', 'E'],
 ['E', 'E', 'E', 'E', 'E']]
updateBoard(board,[3,0])

[['B', 1, 'E', 'E', 'E'],
 ['B', 2, 'M', 'E', 'E'],
 ['B', 2, 'M', 'E', 'E'],
 ['B', 1, 'E', 'E', 'E']]

In [232]:
# 130. Surrounded Regions
# 从边界出发，如果为O，则继续往下走，或者可以将所有满足条件的存入队列然后BFS左边或DFS右边推出
# 可以用栈队列求解DFS/BFS，可以用迭代求解
def solve(board):#用迭代
    """
    :type board: List[List[str]]
    :rtype: None Do not return anything, modify board in-place instead.
    """
    m,n = len(board),len(board[0])
    visited = [[False for i in range(n)] for _ in range(m)]


    def dfs(i,j):
#         print(i,j,visited[i][j])
        if not visited[i][j]:
            if board[i][j]=='O':
                visited[i][j]=True
                for point in ([i-1,j],[i+1,j],[i,j-1],[i,j+1]):
                    x,y = point[0],point[1]
                    if 0<=x<m and 0<=y<n:
                        dfs(x,y)
    for i in [0,m-1]:
        for j in range(n):
            dfs(i,j)
    for i in range(m):
        for j in [0,n-1]:
            dfs(i,j)

    for i in range(1,m-1):
        for j in range(1,n-1):
            if board[i][j]=='O' and not visited[i][j]:
                board[i][j]='X'
    return board

def solve(board):#用栈/队列
    """
    :type board: List[List[str]]
    :rtype: None Do not return anything, modify board in-place instead.
    """
    m,n = len(board),len(board[0])
    queue = collections.deque([])
    for i in [0,m-1]:
        for j in range(n):
            if board[i][j]=='O':
                queue.append((i,j))
    for i in range(m):
        for j in [0,n-1]:
            if board[i][j]=='O':
                queue.append((i,j))
    
    while queue:
        x,y=queue.popleft()# BFS
        if 0<=x<m and 0<=y<n and board[x][y]=='O':
            queue.append((x-1,y));queue.append((x+1,y));queue.append((x,y-1));queue.append((x,y+1))
            board[x][y]='D'
#         x,y=queue.pop()#DFS
#         if 0<=x<m and 0<=y<n and board[x][y]=='O':
#             queue.append((x-1,y));queue.append((x+1,y));queue.append((x,y-1));queue.append((x,y+1))
#             board[x][y]='D'
            
    for i in range(1,m):
        for j in range(1,n):
            if board[i][j]=='O':
                board[i][j]='X'
            if board[i][j]=='D':
                board[i][j]='O'
    return board

solve([["X","X","X","X","X"],["X","O","O","O","O"],["X","X","O","X","X"],["X","O","X","X","X"]])

In [319]:
# 127. Word Ladder
# 考虑到词库可能很长，要在原单词基础上人为修改，而不是在词库中每个单词比对，不同层次放在一起，为避免混淆，将每个元素添加一个元素表示层次
def ladderLength(beginWord, endWord, wordList):
    """
    :type beginWord: str
    :type endWord: str
    :type wordList: List[str]
    :rtype: int
    """
    def match(s1, s2):
        ok = False
        for c1, c2 in zip(s1, s2):
            if c1 != c2:
                if ok:
                    return False
                else:
                    ok = True
        return ok
    
    wordList = set(wordList)
    queue = {i for i in wordList if match(beginWord,i)}
    prev = set()
    res = 1
    while queue:
        newset = set()
        res = res+1
        for i in queue:
            if i==endWord:
                return res
            for j in wordList:
                if j not in queue and j not in prev and match(i,j):
                    newset.add(j)
        prev = prev|queue
        queue = newset
    return 0

def ladderLength(beginWord, endWord, wordList):
    wordList = set(wordList)
    queue = collections.deque([[beginWord, 1]])
    while queue:
        word, length = queue.popleft()
        if word == endWord:
            return length
        for i in range(len(word)):
            for c in 'abcdefghijklmnopqrstuvwxyz':
                next_word = word[:i] + c + word[i+1:]
                if next_word in wordList:
                    wordList.remove(next_word)
                    queue.append([next_word, length + 1])
    return 0

wordList = ["ricky","grind","cubic","panic","lover","farce","gofer","sales","flint","omens","lipid","briny","cloth","anted","slime","oaten","harsh","touts","stoop","cabal","lazed","elton","skunk","nicer","pesky","kusch","bused","kinda","tunis","enjoy","aches","prowl","babar","rooms","burst","slush","pines","urine","pinky","bayed","mania","light","flare","wares","women","verne","moron","shine","bluer","zeros","bleak","brief","tamra","vasts","jamie","lairs","penal","worst","yowls","pills","taros","addle","alyce","creep","saber","floyd","cures","soggy","vexed","vilma","cabby","verde","euler","cling","wanna","jenny","donor","stole","sakha","blake","sanes","riffs","forge","horus","sered","piked","prosy","wases","glove","onset","spake","benin","talks","sites","biers","wendy","dante","allan","haven","nears","shaka","sloth","perky","spear","spend","clint","dears","sadly","units","vista","hinds","marat","natal","least","bough","pales","boole","ditch","greys","slunk","bitch","belts","sense","skits","monty","yawns","music","hails","alien","gibes","lille","spacy","argot","wasps","drubs","poops","bella","clone","beast","emend","iring","start","darla","bells","cults","dhaka","sniff","seers","bantu","pages","fever","tacky","hoses","strop","climb","pairs","later","grant","raven","stael","drips","lucid","awing","dines","balms","della","galen","toned","snips","shady","chili","fears","nurse","joint","plump","micky","lions","jamal","queer","ruins","frats","spoof","semen","pulps","oldie","coors","rhone","papal","seals","spans","scaly","sieve","klaus","drums","tided","needs","rider","lures","treks","hares","liner","hokey","boots","primp","laval","limes","putts","fonda","damon","pikes","hobbs","specs","greet","ketch","braid","purer","tsars","berne","tarts","clean","grate","trips","chefs","timex","vicky","pares","price","every","beret","vices","jodie","fanny","mails","built","bossy","farms","pubic","gongs","magma","quads","shell","jocks","woods","waded","parka","jells","worse","diner","risks","bliss","bryan","terse","crier","incur","murky","gamed","edges","keens","bread","raced","vetch","glint","zions","porno","sizes","mends","ached","allie","bands","plank","forth","fuels","rhyme","wimpy","peels","foggy","wings","frill","edgar","slave","lotus","point","hints","germs","clung","limed","loafs","realm","myron","loopy","plush","volts","bimbo","smash","windy","sours","choke","karin","boast","whirr","tiber","dimes","basel","cutes","pinto","troll","thumb","decor","craft","tared","split","josue","tramp","screw","label","lenny","apses","slept","sikhs","child","bouts","cites","swipe","lurks","seeds","fists","hoard","steed","reams","spoil","diego","peale","bevel","flags","mazes","quart","snipe","latch","lards","acted","falls","busby","holed","mummy","wrong","wipes","carlo","leers","wails","night","pasty","eater","flunk","vedas","curse","tyros","mirth","jacky","butte","wired","fixes","tares","vague","roved","stove","swoon","scour","coked","marge","cants","comic","corns","zilch","typos","lives","truer","comma","gaily","teals","witty","hyper","croat","sways","tills","hones","dowel","llano","clefs","fores","cinch","brock","vichy","bleed","nuder","hoyle","slams","macro","arabs","tauts","eager","croak","scoop","crime","lurch","weals","fates","clipt","teens","bulls","domed","ghana","culls","frame","hanky","jared","swain","truss","drank","lobby","lumps","pansy","whews","saris","trite","weeps","dozes","jeans","flood","chimu","foxes","gelds","sects","scoff","poses","mares","famed","peers","hells","laked","zests","wring","steal","snoot","yodel","scamp","ellis","bandy","marry","jives","vises","blurb","relay","patch","haley","cubit","heine","place","touch","grain","gerry","badly","hooke","fuchs","savor","apron","judge","loren","britt","smith","tammy","altar","duels","huber","baton","dived","apace","sedan","basts","clark","mired","perch","hulks","jolly","welts","quack","spore","alums","shave","singe","lanny","dread","profs","skeet","flout","darin","newed","steer","taine","salvo","mites","rules","crash","thorn","olive","saves","yawed","pique","salon","ovens","dusty","janie","elise","carve","winds","abash","cheep","strap","fared","discs","poxed","hoots","catch","combo","maize","repay","mario","snuff","delve","cored","bards","sudan","shuns","yukon","jowls","wayne","torus","gales","creek","prove","needy","wisps","terri","ranks","books","dicky","tapes","aping","padre","roads","nines","seats","flats","rains","moira","basic","loves","pulls","tough","gills","codes","chest","teeny","jolts","woody","flame","asked","dulls","hotly","glare","mucky","spite","flake","vines","lindy","butts","froth","beeps","sills","bunny","flied","shaun","mawed","velds","voled","doily","patel","snake","thigh","adler","calks","desks","janus","spunk","baled","match","strip","hosed","nippy","wrest","whams","calfs","sleet","wives","boars","chain","table","duked","riped","edens","galas","huffs","biddy","claps","aleut","yucks","bangs","quids","glenn","evert","drunk","lusts","senna","slate","manet","roted","sleep","loxes","fluky","fence","clamp","doted","broad","sager","spark","belch","mandy","deana","beyer","hoist","leafy","levee","libel","tonic","aloes","steam","skews","tides","stall","rifts","saxon","mavis","asama","might","dotes","tangs","wroth","kited","salad","liens","clink","glows","balky","taffy","sided","sworn","oasis","tenth","blurt","tower","often","walsh","sonny","andes","slump","scans","boded","chive","finer","ponce","prune","sloes","dined","chums","dingo","harte","ahead","event","freer","heart","fetch","sated","soapy","skins","royal","cuter","loire","minot","aisle","horny","slued","panel","eight","snoop","pries","clive","pored","wrist","piped","daren","cells","parks","slugs","cubed","highs","booze","weary","stain","hoped","finny","weeds","fetid","racer","tasks","right","saint","shahs","basis","refer","chart","seize","lulls","slant","belay","clots","jinny","tours","modes","gloat","dunks","flute","conch","marts","aglow","gayer","lazes","dicks","chime","bears","sharp","hatch","forms","terry","gouda","thins","janet","tonya","axons","sewed","danny","rowdy","dolts","hurry","opine","fifty","noisy","spiky","humid","verna","poles","jayne","pecos","hooky","haney","shams","snots","sally","ruder","tempe","plunk","shaft","scows","essie","dated","fleet","spate","bunin","hikes","sodas","filly","thyme","fiefs","perks","chary","kiths","lidia","lefty","wolff","withe","three","crawl","wotan","brown","japed","tolls","taken","threw","crave","clash","layer","tends","notes","fudge","musky","bawdy","aline","matts","shirr","balks","stash","wicks","crepe","foods","fares","rotes","party","petty","press","dolly","mangy","leeks","silly","leant","nooks","chapt","loose","caged","wages","grist","alert","sheri","moody","tamps","hefts","souls","rubes","rolex","skulk","veeps","nonce","state","level","whirl","bight","grits","reset","faked","spiny","mixes","hunks","major","missy","arius","damns","fitly","caped","mucus","trace","surat","lloyd","furry","colin","texts","livia","reply","twill","ships","peons","shear","norms","jumbo","bring","masks","zippy","brine","dorks","roded","sinks","river","wolfs","strew","myths","pulpy","prank","veins","flues","minus","phone","banns","spell","burro","brags","boyle","lambs","sides","knees","clews","aired","skirt","heavy","dimer","bombs","scums","hayes","chaps","snugs","dusky","loxed","ellen","while","swank","track","minim","wiled","hazed","roofs","cantu","sorry","roach","loser","brass","stint","jerks","dirks","emory","campy","poise","sexed","gamer","catty","comte","bilbo","fasts","ledge","drier","idles","doors","waged","rizal","pured","weirs","crisp","tasty","sored","palmy","parts","ethel","unify","crows","crest","udder","delis","punks","dowse","totes","emile","coded","shops","poppa","pours","gushy","tiffs","shads","birds","coils","areas","boons","hulls","alter","lobes","pleat","depth","fires","pones","serra","sweat","kline","malay","ruled","calve","tired","drabs","tubed","wryer","slung","union","sonya","aided","hewed","dicey","grids","nixed","whits","mills","buffs","yucky","drops","ready","yuppy","tweet","napes","cadre","teach","rasps","dowdy","hoary","canto","posed","dumbo","kooks","reese","snaky","binge","byron","phony","safer","friar","novel","scale","huron","adorn","carla","fauna","myers","hobby","purse","flesh","smock","along","boils","pails","times","panza","lodge","clubs","colby","great","thing","peaks","diana","vance","whets","bergs","sling","spade","soaks","beach","traps","aspen","romps","boxed","fakir","weave","nerds","swazi","dotty","curls","diver","jonas","waite","verbs","yeast","lapel","barth","soars","hooks","taxed","slews","gouge","slags","chang","chafe","saved","josie","syncs","fonds","anion","actor","seems","pyrex","isiah","glued","groin","goren","waxes","tonia","whine","scads","knelt","teaks","satan","tromp","spats","merry","wordy","stake","gland","canal","donna","lends","filed","sacks","shied","moors","paths","older","pooch","balsa","riced","facet","decaf","attic","elder","akron","chomp","chump","picky","money","sheer","bolls","crabs","dorms","water","veers","tease","dummy","dumbs","lethe","halls","rifer","demon","fucks","whips","plops","fuses","focal","taces","snout","edict","flush","burps","dawes","lorry","spews","sprat","click","deann","sited","aunts","quips","godly","pupil","nanny","funks","shoon","aimed","stacy","helms","mints","banks","pinch","local","twine","pacts","deers","halos","slink","preys","potty","ruffs","pusan","suits","finks","slash","prods","dense","edsel","heeds","palls","slats","snits","mower","rares","ailed","rouge","ellie","gated","lyons","duded","links","oaths","letha","kicks","firms","gravy","month","kongo","mused","ducal","toted","vocal","disks","spied","studs","macao","erick","coupe","starr","reaps","decoy","rayon","nicks","breed","cosby","haunt","typed","plain","trays","muled","saith","drano","cower","snows","buses","jewry","argus","doers","flays","swish","resin","boobs","sicks","spies",\
            "bails","wowed","mabel","check","vapid","bacon","wilda","ollie","loony","irked","fraud","doles","facts","lists","gazed","furls","sunks","stows","wilde","brick","bowed","guise","suing","gates","niter","heros","hyped","clomp","never","lolls","rangy","paddy","chant","casts","terns","tunas","poker","scary","maims","saran","devon","tripe","lingo","paler","coped","bride","voted","dodge","gross","curds","sames","those","tithe","steep","flaks","close","swops","stare","notch","prays","roles","crush","feuds","nudge","baned","brake","plans","weepy","dazed","jenna","weiss","tomes","stews","whist","gibed","death","clank","cover","peeks","quick","abler","daddy","calls","scald","lilia","flask","cheer","grabs","megan","canes","jules","blots","mossy","begun","freak","caved","hello","hades","theed","wards","darcy","malta","peter","whorl","break","downs","odder","hoofs","kiddo","macho","fords","liked","flees","swing","elect","hoods","pluck","brook","astir","bland","sward","modal","flown","ahmad","waled","craps","cools","roods","hided","plath","kings","grips","gives","gnats","tabby","gauls","think","bully","fogey","sawed","lints","pushy","banes","drake","trail","moral","daley","balds","chugs","geeky","darts","soddy","haves","opens","rends","buggy","moles","freud","gored","shock","angus","puree","raves","johns","armed","packs","minis","reich","slots","totem","clown","popes","brute","hedge","latin","stoke","blend","pease","rubik","greer","hindi","betsy","flows","funky","kelli","humps","chewy","welds","scowl","yells","cough","sasha","sheaf","jokes","coast","words","irate","hales","camry","spits","burma","rhine","bends","spill","stubs","power","voles","learn","knoll","style","twila","drove","dacca","sheen","papas","shale","jones","duped","tunny","mouse","floss","corks","skims","swaps","inned","boxer","synch","skies","strep","bucks","belau","lower","flaky","quill","aural","rufus","floes","pokes","sends","sates","dally","boyer","hurts","foyer","gowns","torch","luria","fangs","moats","heinz","bolts","filet","firth","begot","argue","youth","chimp","frogs","kraft","smite","loges","loons","spine","domes","pokey","timur","noddy","doggy","wades","lanes","hence","louts","turks","lurid","goths","moist","bated","giles","stood","winos","shins","potts","brant","vised","alice","rosie","dents","babes","softy","decay","meats","tanya","rusks","pasts","karat","nuked","gorge","kinks","skull","noyce","aimee","watch","cleat","stuck","china","testy","doses","safes","stage","bayes","twins","limps","denis","chars","flaps","paces","abase","grays","deans","maria","asset","smuts","serbs","whigs","vases","robyn","girls","pents","alike","nodal","molly","swigs","swill","slums","rajah","bleep","beget","thanh","finns","clock","wafts","wafer","spicy","sorer","reach","beats","baker","crown","drugs","daisy","mocks","scots","fests","newer","agate","drift","marta","chino","flirt","homed","bribe","scram","bulks","servo","vesta","divas","preps","naval","tally","shove","ragas","blown","droll","tryst","lucky","leech","lines","sires","pyxed","taper","trump","payee","midge","paris","bored","loads","shuts","lived","swath","snare","boned","scars","aeons","grime","writs","paige","rungs","blent","signs","davis","dials","daubs","rainy","fawns","wrier","golds","wrath","ducks","allow","hosea","spike","meals","haber","muses","timed","broom","burks","louis","gangs","pools","vales","altai","elope","plied","slain","chasm","entry","slide","bawls","title","sings","grief","viola","doyle","peach","davit","bench","devil","latex","miles","pasha","tokes","coves","wheel","tried","verdi","wanda","sivan","prior","fryer","plots","kicky","porch","shill","coats","borne","brink","pawed","erwin","tense","stirs","wends","waxen","carts","smear","rival","scare","phase","bragg","crane","hocks","conan","bests","dares","molls","roots","dunes","slips","waked","fours","bolds","slosh","yemen","poole","solid","ports","fades","legal","cedes","green","curie","seedy","riper","poled","glade","hosts","tools","razes","tarry","muddy","shims","sword","thine","lasts","bloat","soled","tardy","foots","skiff","volta","murks","croci","gooks","gamey","pyxes","poems","kayla","larva","slaps","abuse","pings","plows","geese","minks","derby","super","inked","manic","leaks","flops","lajos","fuzes","swabs","twigs","gummy","pyres","shrew","islet","doled","wooly","lefts","hunts","toast","faith","macaw","sonia","leafs","colas","conks","altos","wiped","scene","boors","patsy","meany","chung","wakes","clear","ropes","tahoe","zones","crate","tombs","nouns","garth","puked","chats","hanks","baked","binds","fully","soaps","newel","yarns","puers","carps","spelt","lully","towed","scabs","prime","blest","patty","silky","abner","temps","lakes","tests","alias","mines","chips","funds","caret","splat","perry","turds","junks","cramp","saned","peary","snarl","fired","stung","nancy","bulge","styli","seams","hived","feast","triad","jaded","elvin","canny","birth","routs","rimed","pusey","laces","taste","basie","malls","shout","prier","prone","finis","claus","loops","heron","frump","spare","menus","ariel","crams","bloom","foxed","moons","mince","mixed","piers","deres","tempt","dryer","atone","heats","dario","hawed","swims","sheet","tasha","dings","clare","aging","daffy","wried","foals","lunar","havel","irony","ronny","naves","selma","gurus","crust","percy","murat","mauro","cowed","clang","biker","harms","barry","thump","crude","ulnae","thong","pager","oases","mered","locke","merle","soave","petal","poser","store","winch","wedge","inlet","nerdy","utter","filth","spray","drape","pukes","ewers","kinds","dates","meier","tammi","spoor","curly","chill","loped","gooey","boles","genet","boost","beets","heath","feeds","growl","livid","midst","rinds","fresh","waxed","yearn","keeps","rimes","naked","flick","plies","deeps","dirty","hefty","messy","hairy","walks","leper","sykes","nerve","rover","jived","brisk","lenin","viper","chuck","sinus","luger","ricks","hying","rusty","kathy","herds","wider","getty","roman","sandy","pends","fezes","trios","bites","pants","bless","diced","earth","shack","hinge","melds","jonah","chose","liver","salts","ratty","ashed","wacky","yokes","wanly","bruce","vowel","black","grail","lungs","arise","gluts","gluey","navel","coyer","ramps","miter","aldan","booth","musty","rills","darns","tined","straw","kerri","hared","lucks","metes","penny","radon","palms","deeds","earls","shard","pried","tampa","blank","gybes","vicki","drool","groom","curer","cubes","riggs","lanky","tuber","caves","acing","golly","hodge","beard","ginny","jibed","fumes","astor","quito","cargo","randi","gawky","zings","blind","dhoti","sneak","fatah","fixer","lapps","cline","grimm","fakes","maine","erika","dealt","mitch","olden","joist","gents","likes","shelf","silts","goats","leads","marin","spire","louie","evans","amuse","belly","nails","snead","model","whats","shari","quote","tacks","nutty","lames","caste","hexes","cooks","miner","shawn","anise","drama","trike","prate","ayers","loans","botch","vests","cilia","ridge","thugs","outed","jails","moped","plead","tunes","nosed","wills","lager","lacks","cried","wince","berle","flaws","boise","tibet","bided","shred","cocky","brice","delta","congo","holly","hicks","wraps","cocks","aisha","heard","cured","sades","horsy","umped","trice","dorky","curve","ferry","haler","ninth","pasta","jason","honer","kevin","males","fowls","awake","pores","meter","skate","drink","pussy","soups","bases","noyes","torts","bogus","still","soupy","dance","worry","eldon","stern","menes","dolls","dumpy","gaunt","grove","coops","mules","berry","sower","roams","brawl","greed","stags","blurs","swift","treed","taney","shame","easel","moves","leger","ville","order","spock","nifty","brian","elias","idler","serve","ashen","bizet","gilts","spook","eaten","pumas","cotes","broke","toxin","groan","laths","joins","spots","hated","tokay","elite","rawer","fiats","cards","sassy","milks","roost","glean","lutes","chins","drown","marks","pined","grace","fifth","lodes","rusts","terms","maxes","savvy","choir","savoy","spoon","halve","chord","hulas","sarah","celia","deems","ninny","wines","boggy","birch","raved","wales","beams","vibes","riots","warty","nigel","askew","faxes","sedge","sheol","pucks","cynic","relax","boers","whims","bents","candy","luann","slogs","bonny","barns","iambs","fused","duffy","guilt","bruin","pawls","penis","poppy","owing","tribe","tuner","moray","timid","ceded","geeks","kites","curio","puffy","perot","caddy","peeve","cause","dills","gavel","manse","joker","lynch","crank","golda","waits","wises","hasty","paves","grown","reedy","crypt","tonne","jerky","axing","swept","posse","rings","staff","tansy","pared","glaze","grebe","gonna","shark","jumps","vials","unset","hires","tying","lured","motes","linen","locks","mamas","nasty","mamie","clout","nader","velma","abate","tight","dales","serer","rives","bales","loamy","warps","plato","hooch","togae","damps","ofter","plumb","fifes","filmy","wiper","chess","lousy","sails","brahe","ounce","flits","hindu","manly","beaux","mimed","liken","forts","jambs","peeps","lelia","brews","handy","lusty","brads","marne","pesos","earle","arson","scout","showy","chile","sumps","hiked","crook","herbs","silks","alamo","mores","dunce","blaze","stank","haste","howls","trots","creon","lisle","pause","hates","mulch","mined","moder","devin","types","cindy","beech","tuned","mowed","pitts","chaos","colds","bidet","tines","sighs","slimy","brain","belle","leery","morse","ruben","prows","frown","disco","regal","oaken","sheds","hives","corny","baser","fated","throe","revel","bores","waved","shits","elvia","ferns","maids","color","coifs","cohan","draft","hmong","alton","stine","cluck","nodes","emily","brave",\
            "blair","blued","dress","bunts","holst","clogs","rally","knack","demos","brady","blues","flash","goofy","blocs","diane","colic","smile","yules","foamy","splay","bilge","faker","foils","condo","knell","crack","gallo","purls","auras","cakes","doves","joust","aides","lades","muggy","tanks","middy","tarps","slack","capet","frays","donny","venal","yeats","misty","denim","glass","nudes","seeps","gibbs","blows","bobbi","shane","yards","pimps","clued","quiet","witch","boxes","prawn","kerry","torah","kinko","dingy","emote","honor","jelly","grins","trope","vined","bagel","arden","rapid","paged","loved","agape","mural","budge","ticks","suers","wendi","slice","salve","robin","bleat","batik","myles","teddy","flatt","puppy","gelid","largo","attar","polls","glide","serum","fundy","sucks","shalt","sewer","wreak","dames","fonts","toxic","hines","wormy","grass","louse","bowls","crass","benny","moire","margo","golfs","smart","roxie","wight","reign","dairy","clops","paled","oddly","sappy","flair","shown","bulgy","benet","larch","curry","gulfs","fends","lunch","dukes","doris","spoke","coins","manna","conga","jinns","eases","dunno","tisha","swore","rhino","calms","irvin","clans","gully","liege","mains","besot","serge","being","welch","wombs","draco","lynda","forty","mumps","bloch","ogden","knits","fussy","alder","danes","loyal","valet","wooer","quire","liefs","shana","toyed","forks","gages","slims","cloys","yates","rails","sheep","nacho","divan","honks","stone","snack","added","basal","hasps","focus","alone","laxes","arose","lamed","wrapt","frail","clams","plait","hover","tacos","mooch","fault","teeth","marva","mucks","tread","waves","purim","boron","horde","smack","bongo","monte","swirl","deals","mikes","scold","muter","sties","lawns","fluke","jilts","meuse","fives","sulky","molds","snore","timmy","ditty","gasps","kills","carey","jawed","byers","tommy","homer","hexed","dumas","given","mewls","smelt","weird","speck","merck","keats","draws","trent","agave","wells","chews","blabs","roves","grieg","evens","alive","mulls","cared","garbo","fined","happy","trued","rodes","thurs","cadet","alvin","busch","moths","guild","staci","lever","widen","props","hussy","lamer","riley","bauer","chirp","rants","poxes","shyer","pelts","funny","slits","tinge","ramos","shift","caper","credo","renal","veils","covey","elmer","mated","tykes","wooed","briar","gears","foley","shoes","decry","hypes","dells","wilds","runts","wilts","white","easts","comer","sammy","lochs","favor","lance","dawns","bushy","muted","elsie","creel","pocks","tenet","cagey","rides","socks","ogled","soils","sofas","janna","exile","barks","frank","takes","zooms","hakes","sagan","scull","heaps","augur","pouch","blare","bulbs","wryly","homey","tubas","limbo","hardy","hoagy","minds","bared","gabby","bilks","float","limns","clasp","laura","range","brush","tummy","kilts","cooed","worms","leary","feats","robes","suite","veals","bosch","moans","dozen","rarer","slyer","cabin","craze","sweet","talon","treat","yanks","react","creed","eliza","sluts","cruet","hafts","noise","seder","flies","weeks","venus","backs","eider","uriel","vouch","robed","hacks","perth","shiny","stilt","torte","throb","merer","twits","reeds","shawl","clara","slurs","mixer","newts","fried","woolf","swoop","kaaba","oozed","mayer","caned","laius","lunge","chits","kenny","lifts","mafia","sowed","piled","stein","whack","colts","warms","cleft","girds","seeks","poets","angel","trade","parsi","tiers","rojas","vexes","bryce","moots","grunt","drain","lumpy","stabs","poohs","leapt","polly","cuffs","giddy","towns","dacha","quoth","provo","dilly","carly","mewed","tzars","crock","toked","speak","mayas","pssts","ocher","motel","vogue","camps","tharp","taunt","drone","taint","badge","scott","scats","bakes","antes","gruel","snort","capes","plate","folly","adobe","yours","papaw","hench","moods","clunk","chevy","tomas","narcs","vonda","wiles","prigs","chock","laser","viced","stiff","rouse","helps","knead","gazer","blade","tumid","avail","anger","egged","guide","goads","rabin","toddy","gulps","flank","brats","pedal","junky","marco","tinny","tires","flier","satin","darth","paley","gumbo","rared","muffs","rower","prude","frees","quays","homes","munch","beefs","leash","aston","colon","finch","bogey","leaps","tempo","posts","lined","gapes","locus","maori","nixes","liven","songs","opted","babel","wader","barer","farts","lisps","koran","lathe","trill","smirk","mamma","viler","scurf","ravel","brigs","cooky","sachs","fulls","goals","turfs","norse","hauls","cores","fairy","pluto","kneed","cheek","pangs","risen","czars","milne","cribs","genes","wefts","vents","sages","seres","owens","wiley","flume","haded","auger","tatty","onion","cater","wolfe","magic","bodes","gulls","gazes","dandy","snags","rowed","quell","spurn","shore","veldt","turns","slavs","coach","stalk","snuck","piles","orate","joyed","daily","crone","wager","solos","earns","stark","lauds","kasey","villa","gnaws","scent","wears","fains","laced","tamer","pipes","plant","lorie","rivet","tamed","cozen","theme","lifer","sunny","shags","flack","gassy","eased","jeeps","shire","fargo","timer","brash","behan","basin","volga","krone","swiss","docks","booed","ebert","gusty","delay","oared","grady","buick","curbs","crete","lucas","strum","besom","gorse","troth","donne","chink","faced","ahmed","texas","longs","aloud","bethe","cacao","hilda","eagle","karyn","harks","adder","verse","drays","cello","taped","snide","taxis","kinky","penes","wicca","sonja","aways","dyers","bolas","elfin","slope","lamps","hutch","lobed","baaed","masts","ashes","ionic","joyce","payed","brays","malts","dregs","leaky","runny","fecal","woven","hurls","jorge","henna","dolby","booty","brett","dykes","rural","fight","feels","flogs","brunt","preen","elvis","dopey","gripe","garry","gamma","fling","space","mange","storm","arron","hairs","rogue","repel","elgar","ruddy","cross","medan","loses","howdy","foams","piker","halts","jewel","avery","stool","cruel","cases","ruses","cathy","harem","flour","meted","faces","hobos","charm","jamar","cameo","crape","hooey","reefs","denny","mitts","sores","smoky","nopes","sooty","twirl","toads","vader","julep","licks","arias","wrote","north","bunks","heady","batch","snaps","claws","fouls","faded","beans","wimps","idled","pulse","goons","noose","vowed","ronda","rajas","roast","allah","punic","slows","hours","metal","slier","meaty","hanna","curvy","mussy","truth","troys","block","reels","print","miffs","busts","bytes","cream","otter","grads","siren","kilos","dross","batty","debts","sully","bares","baggy","hippy","berth","gorky","argon","wacko","harry","smoke","fails","perms","score","steps","unity","couch","kelly","rumps","fines","mouth","broth","knows","becky","quits","lauri","trust","grows","logos","apter","burrs","zincs","buyer","bayer","moose","overt","croon","ousts","lands","lithe","poach","jamel","waive","wiser","surly","works","paine","medal","glads","gybed","paint","lorre","meant","smugs","bryon","jinni","sever","viols","flubs","melts","heads","peals","aiken","named","teary","yalta","styes","heist","bongs","slops","pouts","grape","belie","cloak","rocks","scone","lydia","goofs","rents","drive","crony","orlon","narks","plays","blips","pence","march","alger","baste","acorn","billy","croce","boone","aaron","slobs","idyls","irwin","elves","stoat","doing","globe","verve","icons","trial","olsen","pecks","there","blame","tilde","milky","sells","tangy","wrack","fills","lofty","truce","quark","delia","stowe","marty","overs","putty","coral","swine","stats","swags","weans","spout","bulky","farsi","brest","gleam","beaks","coons","hater","peony","huffy","exert","clips","riven","payer","doped","salas","meyer","dryad","thuds","tilts","quilt","jetty","brood","gulch","corps","tunic","hubby","slang","wreck","purrs","punch","drags","chide","sulks","tints","huger","roped","dopes","booby","rosin","outer","gusto","tents","elude","brows","lease","ceres","laxer","worth","necks","races","corey","trait","stuns","soles","teems","scrip","privy","sight","minor","alisa","stray","spank","cress","nukes","rises","gusts","aurae","karma","icing","prose","biked","grand","grasp","skein","shaky","clump","rummy","stock","twain","zoned","offed","ghats","mover","randy","vault","craws","thees","salem","downy","sangs","chore","cited","grave","spinx","erica","raspy","dying","skips","clerk","paste","moved","rooks","intel","moses","avers","staid","yawls","blast","lyres","monks","gaits","floor","saner","waver","assam","infer","wands","bunch","dryly","weedy","honey","baths","leach","shorn","shows","dream","value","dooms","spiro","raped","shook","stead","moran","ditto","loots","tapir","looms","clove","stops","pinks","soppy","ripen","wench","shone","bauds","doric","leans","nadia","cries","camus","boozy","maris","fools","morns","bides","greek","gauss","roget","lamar","hazes","beefy","dupes","refed","felts","larry","guile","ables","wants","warns","toils","bathe","edger","paced","rinks","shoos","erich","whore","tiger","jumpy","lamas","stack","among","punts","scalp","alloy","solon","quite","comas","whole","parse","tries","reeve","tiled","deena","roomy","rodin","aster","twice","musts","globs","parch","drawn","filch","bonds","tells","droop","janis","holds","scant","lopes","based","keven","whiny","aspic","gains","franz","jerri","steel","rowel","vends","yelps","begin","logic","tress","sunni","going","barge","blood","burns","basks","waifs","bones","skill","hewer","burly","clime","eking","withs","capek","berta","cheap","films","scoot","tweed","sizer","wheat","acton","flung","ponds","tracy","fiver","berra","roger","mutes","burke","miked","valve","whisk","runes","parry","toots","japes","roars","rough","irons","romeo","cages","reeks","cigar","saiph","dully","hangs","chops","rolls","prick","acuff","spent","sulla","train","swell","frets","names","anita","crazy","sixth","blunt","fewer","large",\
            "brand","slick","spitz","rears","ogres","toffy","yolks","flock","gnawn","eries","blink","skier","feted","tones","snail","ether","barbs","noses","hears","upset","awash","cloud","trunk","degas","dungs","rated","shall","yeahs","coven","sands","susan","fable","gunny","began","serfs","balls","dinky","madge","prong","spilt","lilly","brawn","comet","spins","raids","dries","sorts","makes","mason","mayra","royce","stout","mealy","pagan","nasal","folds","libby","coups","photo","mosey","amens","speed","lords","board","fetal","lagos","scope","raked","bonus","mutts","willy","sport","bingo","thant","araby","bette","rebel","gases","small","humus","grosz","beset","slays","steve","scrap","blahs","south","pride","heels","tubes","beady","lacey","genus","mauls","vying","spice","sexes","ester","drams","today","comae","under","jests","direr","yoked","tempi","early","boats","jesus","warts","guppy","gilda","quota","token","edwin","ringo","gaped","lemon","hurst","manor","arrow","mists","prize","silas","blobs","diets","ervin","stony","buddy","bates","rabid","ducat","ewing","jaunt","beads","doyen","blush","thoth","tiles","piper","short","peron","alley","decks","shunt","whirs","cushy","roils","betty","plugs","woken","jibes","foray","merak","ruing","becks","whale","shoot","dwelt","spawn","fairs","dozed","celts","blond","tikes","sabin","feint","vamps","cokes","willa","slues","bills","force","curst","yokel","surer","miler","fices","arced","douse","hilly","lucio","tongs","togas","minty","sagas","pates","welsh","bruno","decal","elate","linux","gyros","pryor","mousy","pains","shake","spica","pupal","probe","mount","shirk","purus","kilns","rests","graze","hague","spuds","sweep","momma","burch","maces","samar","brace","riser","booms","build","camel","flyer","synge","sauna","tonga","tings","promo","hides","clair","elisa","bower","reins","diann","lubed","nulls","picks","laban","milch","buber","stomp","bosom","lying","haled","avert","wries","macon","skids","fumed","ogles","clods","antic","nosey","crimp","purge","mommy","cased","taxes","covet","clack","butch","panty","lents","machs","exude","tooth","adore","shuck","asses","after","terra","dices","aryan","regor","romes","stile","cairo","maura","flail","eaves","estes","sousa","visas","baron","civet","kitty","freed","ralph","tango","gawks","cheat","study","fancy","fiber","musks","souse","brims","claim","bikes","venue","sired","thymi","rivas","skimp","pleas","woman","gimpy","cawed","minos","pints","knock","poked","bowen","risky","towel","oinks","linus","heals","pears","codas","inner","pitch","harpy","niger","madly","bumpy","stair","files","nobel","celli","spars","jades","balmy","kooky","plums","trues","gloss","trims","daunt","tubby","dared","wadis","smell","darby","stink","drill","dover","ruler","laden","dikes","layla","fells","maker","joked","horns","these","baize","spahn","whens","edged","mushy","plume","tucks","spurs","husky","dried","bigot","pupas","drily","aware","hagar","newly","knots","pratt","feces","sabik","watts","cooke","riles","seamy","fleas","dusts","barfs","roans","pawns","vivid","kirks","tania","feral","tubae","horne","aries","brits","combs","chunk","stork","waned","texan","elide","glens","emery","autos","trams","dosed","cheri","baits","jacks","whose","fazed","matte","swans","maxed","write","spays","orion","traci","horse","stars","strut","goods","verge","scuff","award","dives","wires","burnt","dimly","sleds","mayan","biped","quirk","sofia","slabs","waste","robby","mayor","fatty","items","bowel","mires","swarm","route","swash","sooth","paved","steak","upend","sough","throw","perts","stave","carry","burgs","hilts","plane","toady","nadir","stick","foist","gnarl","spain","enter","sises","story","scarf","ryder","glums","nappy","sixes","honed","marcy","offer","kneel","leeds","lites","voter","vince","bursa","heave","roses","trees","argos","leann","grimy","zelma","crick","tract","flips","folks","smote","brier","moore","goose","baden","riled","looks","sober","tusks","house","acmes","lubes","chows","neath","vivas","defer","allay","casey","kmart","pests","proms","eying","cider","leave","shush","shots","karla","scorn","gifts","sneer","mercy","copes","faxed","spurt","monet","awoke","rocky","share","gores","drawl","tears","mooed","nones","wined","wrens","modem","beria","hovel","retch","mates","hands","stymy","peace","carat","coots","hotel","karen","hayed","mamet","cuing","paper","rages","suave","reuse","auden","costs","loner","rapes","hazel","rites","brent","pumps","dutch","puffs","noons","grams","teats","cease","honda","pricy","forgo","fleck","hired","silos","merge","rafts","halon","larks","deere","jello","cunts","sifts","boner","morin","mimes","bungs","marie","harts","snobs","sonic","hippo","comes","crops","mango","wrung","garbs","natty","cents","fitch","moldy","adams","sorta","coeds","gilds","kiddy","nervy","slurp","ramon","fuzed","hiker","winks","vanes","goody","hawks","crowd","bract","marla","limbs","solve","gloom","sloop","eaton","memos","tames","heirs","berms","wanes","faint","numbs","holes","grubs","rakes","waist","miser","stays","antis","marsh","skyed","payne","champ","jimmy","clues","fatal","shoed","freon","lopez","snowy","loins","stale","thank","reads","isles","grill","align","saxes","rubin","rigel","walls","beers","wispy","topic","alden","anton","ducts","david","duets","fries","oiled","waken","allot","swats","woozy","tuxes","inter","dunne","known","axles","graph","bumps","jerry","hitch","crews","lucia","banal","grope","valid","meres","thick","lofts","chaff","taker","glues","snubs","trawl","keels","liker","stand","harps","casks","nelly","debby","panes","dumps","norma","racks","scams","forte","dwell","dudes","hypos","sissy","swamp","faust","slake","maven","lowed","lilts","bobby","gorey","swear","nests","marci","palsy","siege","oozes","rates","stunt","herod","wilma","other","girts","conic","goner","peppy","class","sized","games","snell","newsy","amend","solis","duane","troop","linda","tails","woofs","scuds","shies","patti","stunk","acres","tevet","allen","carpi","meets","trend","salty","galls","crept","toner","panda","cohen","chase","james","bravo","styed","coals","oates","swami","staph","frisk","cares","cords","stems","razed","since","mopes","rices","junes","raged","liter","manes","rearm","naive","tyree","medic","laded","pearl","inset","graft","chair","votes","saver","cains","knobs","gamay","hunch","crags","olson","teams","surge","wests","boney","limos","ploys","algae","gaols","caked","molts","glops","tarot","wheal","cysts","husks","vaunt","beaus","fauns","jeers","mitty","stuff","shape","sears","buffy","maced","fazes","vegas","stamp","borer","gaged","shade","finds","frock","plods","skied","stump","ripes","chick","cones","fixed","coled","rodeo","basil","dazes","sting","surfs","mindy","creak","swung","cadge","franc","seven","sices","weest","unite","codex","trick","fusty","plaid","hills","truck","spiel","sleek","anons","pupae","chiba","hoops","trash","noted","boris","dough","shirt","cowls","seine","spool","miens","yummy","grade","proxy","hopes","girth","deter","dowry","aorta","paean","corms","giant","shank","where","means","years","vegan","derek","tales"]

ladderLength(beginWord = "nanny",endWord = "aloud",wordList = wordList)

20

In [2]:
# 126. Word Ladder II
import collections
def findLadders(beginWord, endWord, wordList):
    """
    :type beginWord: str
    :type endWord: str
    :type wordList: List[str]
    :rtype: List[List[str]]
    """
    wordList = set(wordList)
    queue = [[beginWord]]
    while queue:
        newqueue = []
        for words in queue:
            word = words[-1]
            if word == endWord:
                break
            for i in range(len(word)):
                for ch in 'abcdefghijklmnopqrstuvwxyz':
                    next_word = word[:i] + ch + word[i+1:]
                    if next_word in wordList:
                        temp = words +[next_word]
                        newqueue += [temp]
        for i in newqueue:
            if i[-1] in wordList:
                wordList.remove(i[-1])
        if word == endWord:
            break
        queue = newqueue                   
    return [i for i in queue if i[-1]==endWord]

def findLadders(beginWord, endWord, wordList):
    wordList = set(wordList)
    res = []
    layer = {}
    layer[beginWord] = [[beginWord]]

    while layer:
        newlayer = collections.defaultdict(list)
        for w in layer:
            print(w,layer[w])
            if w == endWord: 
                res.extend(k for k in layer[w])
            else:
                for i in range(len(w)):
                    for c in 'abcdefghijklmnopqrstuvwxyz':
                        neww = w[:i]+c+w[i+1:]
                        if neww in wordList:
                            newlayer[neww]+=[j+[neww] for j in layer[w]]

        wordList -= set(newlayer.keys())
        layer = newlayer

    return res
findLadders(beginWord = "hit",endWord = "cog",wordList = ["hot","dot","dog","lot","log","cog"])
# findLadders(beginWord = "nanny",endWord = "aloud",wordList = wordList)

hit [['hit']]
hot [['hit', 'hot']]
dot [['hit', 'hot', 'dot']]
lot [['hit', 'hot', 'lot']]
dog [['hit', 'hot', 'dot', 'dog']]
log [['hit', 'hot', 'lot', 'log']]
cog [['hit', 'hot', 'dot', 'dog', 'cog'], ['hit', 'hot', 'lot', 'log', 'cog']]


[['hit', 'hot', 'dot', 'dog', 'cog'], ['hit', 'hot', 'lot', 'log', 'cog']]

In [474]:
# 417. Pacific Atlantic Water Flow
def pacificAtlantic(matrix):
    """
    :type matrix: List[List[int]]
    :rtype: List[List[int]]
    """
    direction = [(-1,0),(1,0),(0,-1),(0,1)]
    if not matrix or not matrix[0]:
        return []
    m,n = len(matrix),len(matrix[0])
    pacific = [[False for _ in range(n)] for _ in range(m)]
    atlantic = [[False for _ in range(n)] for _ in range(m)]

    queue =[[i,j] for i in range(m) for j in range(n) if i==0 or j==0]
    while queue:
        [x,y]=queue.pop(0)
        pacific[x][y]=True
        for xadd,yadd in direction:
            xnew,ynew =x+xadd,y+yadd
            if xnew<0 or xnew>=m or ynew<0 or ynew>=n or matrix[x][y]>matrix[xnew][ynew] or pacific[xnew][ynew]:
                continue
            pacific[xnew][ynew]=True
            queue.append([xnew,ynew])
    queue =[[i,j] for i in range(m) for j in range(n) if i==m-1 or j==n-1]
    while queue:
        [x,y]=queue.pop(0)
        atlantic[x][y]=True
        for xadd,yadd in direction:
            xnew,ynew =x+xadd,y+yadd
            if xnew<0 or xnew>=m or ynew<0 or ynew>=n or matrix[x][y]>matrix[xnew][ynew] or atlantic[xnew][ynew]:
                continue
            atlantic[xnew][ynew]=True
            queue.append([xnew,ynew])
    return [[i,j] for i in range(m) for j in range(n) if pacific[i][j] and atlantic[i][j]]

In [475]:
pacificAtlantic([[1,2,2,3,5],[3,2,3,4,4],[2,4,5,3,1],[6,7,1,4,5],[5,1,1,2,4]])

[[True, True, True, True, True], [True, True, True, True, True], [True, True, True, False, False], [True, True, False, False, False], [True, False, False, False, False]] [[False, False, False, False, True], [False, False, False, True, True], [False, False, True, True, True], [True, True, True, True, True], [True, True, True, True, True]]


[[0, 4], [1, 3], [1, 4], [2, 2], [3, 0], [3, 1], [4, 0]]

In [440]:
class Solution(object):
    def pacificAtlantic(self, matrix):
        """
        :type matrix: List[List[int]]
        :rtype: List[List[int]]
        """
        if not matrix: return []
        self.directions = [(1,0),(-1,0),(0,1),(0,-1)]
        m = len(matrix)
        n = len(matrix[0])
        p_visited = [[False for _ in range(n)] for _ in range(m)]
        
        a_visited = [[False for _ in range(n)] for _ in range(m)]
        result = []
        
        for i in range(m):
            # p_visited[i][0] = True
            # a_visited[i][n-1] = True
            self.dfs(matrix, i, 0, p_visited, m, n)
            self.dfs(matrix, i, n-1, a_visited, m, n)
        for j in range(n):
            # p_visited[0][j] = True
            # a_visited[m-1][j] = True
            self.dfs(matrix, 0, j, p_visited, m, n)
            self.dfs(matrix, m-1, j, a_visited, m, n)
            
        for i in range(m):
            for j in range(n):
                if p_visited[i][j] and a_visited[i][j]:
                    result.append([i,j])
        return result
                
                
    def dfs(self, matrix, i, j, visited, m, n):
        # when dfs called, meaning its caller already verified this point 
        visited[i][j] = True
        for dir in self.directions:
            x, y = i + dir[0], j + dir[1]
            if x < 0 or x >= m or y < 0 or y >= n or visited[x][y] or matrix[x][y] < matrix[i][j]:
                continue
            self.dfs(matrix, x, y, visited, m, n)

In [664]:
# 787. Cheapest Flights Within K Stops
# 我的答案是错的，暂时不知道错误原因，用DP做很快
def findCheapestPrice(n, flights, src, dst, K):
    nodes = [i for i in range(n)]
    adj = {i: dict() for i in nodes}
    for i in flights:
        adj[i[0]][i[1]]=i[2]
    dis = {i: adj[src].setdefault(i,float('inf')) for i in nodes if i!=src}
    visited = set()
    
    for i in range(K):
        des = [i for i in dis.keys() if dis[i]<float('inf')]
        temp = {}
        for n in des:
            if n!=dst and n not in visited:
                visited.add(n)
                for m in adj[n].keys():
                    if m!=src and dis[n]+adj[n][m]<dis[m]:
                        temp[m]=dis[n]+adj[n][m]
        for m in temp.keys():
            dis[m]=temp[m]
    return dis[dst] if dis[dst]!=float('inf') else -1

def findCheapestPrice(n, flights, src, dst, K):
    prices = [float('inf') for i in range(n)]
    prices[src]=0
    
    for i in range(K+1):
        newprices = prices[:]
        for a,b,cost in flights:
            newprices[b] = min(newprices[b],prices[a]+cost)
        prices = newprices
    
    return prices[dst] if prices[dst]!=float('inf') else 0

695. Max Area of Island

In [696]:
# 695. Max Area of Island
# DFS利用迭代，边界条件
def maxAreaOfIsland(grid):
    """
    :type grid: List[List[int]]
    :rtype: int
    """
    if not grid or not grid[0]:
        return 0
    directions = [(-1,0),(1,0),(0,-1),(0,1)]
    m,n = len(grid),len(grid[0])
    visited = [[False for _ in range(n)] for _ in range(m)]
    
    def find_next(i,j):
        for col in range(j,n):
            if grid[i][col] and not visited[i][col]:
                return i,col
        for row in range(i,m):
            for col in range(n):
                if grid[row][col] and not visited[row][col]:
                    return row,col
        return -1,-1
    
    i,j=find_next(0,0)
    if i==-1:
        return 0
    queue = [(i,j)]
    length = 0
    res = 0
    while queue:
        x,y=queue.pop(0)
        visited[x][y]=True
        length +=1
        for direction in directions:
            xnew = x+direction[0]
            ynew = y+direction[1]
            if 0<=xnew<m and 0<=ynew<n and grid[xnew][ynew] and visited[xnew][ynew] is False:
                queue.append((xnew,ynew))
                visited[xnew][ynew]=True
        if len(queue)==0:
            res = max(res,length)
            i,j = find_next(x,y)
            length = 0
            if i!=-1:
                queue = [(i,j)]
    return res

def maxAreaOfIsland(grid):
    m, n = len(grid), len(grid[0])

    def dfs(i, j):
        if 0 <= i < m and 0 <= j < n and grid[i][j]:
            grid[i][j] = 0
            return 1 + dfs(i - 1, j) + dfs(i, j + 1) + dfs(i + 1, j) + dfs(i, j - 1)
        return 0

    areas = [dfs(i, j) for i in range(m) for j in range(n) if grid[i][j]]
    return max(areas) if areas else 0

In [700]:
maxAreaOfIsland([[0,0,1,0,0,1]])

1

In [8]:
# 576. Out of Boundary Paths
def findPaths(m, n, N, i, j):
    """
    :type m: int
    :type n: int
    :type N: int
    :type i: int
    :type j: int
    :rtype: int
    """
    directions = [(-1,0),(1,0),(0,-1),(0,1)]
    ans = 0
    def dfs(x,y,counts,N):
        if N==0:
            return counts
        for direction in directions:
            xnew,ynew=x+direction[0],y+direction[1]
            if xnew==-1 or xnew==m or ynew==-1 or ynew==n:
                counts +=1
            else:
                counts += dfs(xnew,ynew,0,N-1)
        return counts%(10**9 + 7)

    return dfs(i,j,ans,N)

def findPaths(m,n,N,i,j):
    mod = 10**9 + 7
    Q = collections.deque([(i,j,0)])
    res = 0
    while Q:
        x,y,step = Q.popleft()
        if step > N: break
        if 0<=x<m and 0<=y<n:
            Q.append((x+1,y,step+1))
            Q.append((x-1,y,step+1))
            Q.append((x,y+1,step+1))
            Q.append((x,y-1,step+1))
        else:
            res += 1
    return res % mod

findPaths(m = 10, n =12, N = 30, i = 5, j = 1)

In [2]:
# 79. Word Search
# 二维矩阵中存在特定的词汇（路径）
def exist(board, word):
    if not board:
        return False
    directions = [[-1,0],[1,0],[0,-1],[0,1]]
    m,n = len(board),len(board[0])
    def dfs(i,j,k,visited=set()):
        if k==len(word)-1:
            return True
        k=k+1
        temp = False
        for direction in directions:
            x,y = i+direction[0],j+direction[1]
            if 0<=x<m and 0<=y<n:
                if (x,y) not in visited and board[x][y]==word[k]:
                    if dfs(x,y,k,visited|{(i,j)}):
                        return True
#                     temp =temp|dfs(x,y,k,visited)
#                     if temp:
#                         visited.add((i,j))
#                         return True
        return False
    for i in range(m):
        for j in range(n):
            if board[i][j]==word[0]:
                if dfs(i,j,k=0,visited=set()):
                    return True
    return False
exist([['A','B','C','E'],['S','F','C','S'],['A','D','E','E']],'ABCESCEE')

True

In [1]:
# 416. Partition Equal Subset Sum
def canPartition(nums):
    """
    :type nums: List[int]
    :rtype: bool
    """
    def exist(d,nums):#nums中是否能凑出这个数
        print(d,nums,d in nums)
        if d in nums or d==0:
            return True
        elif (d!=0 and len(nums)==0) or d<min(nums):
            return False
        else:
            ans=exist(d-nums[0],nums[1:])
            if ans is True:
                return True
            
    if sum(nums)%2==1 or max(nums)>sum(nums)/2:
        return False
    diff=sum(nums)//2-max(nums)#需要补的数
    nums.remove(max(nums))
    if diff==0:#补的数为0
        return True
    print(diff,nums)
    for i in range(len(nums)):#一个个元素判断，是否需要其构成那个差值,如果实在找不到，跳过
        print('i',i,nums[i])
        if exist(diff-nums[i],nums[i+1:]):
            return True
    return False

# def canPartition(nums):#非常慢
#     possible_sums = {0}
#     for n in nums:
#         possible_sums.update({(v + n) for v in possible_sums})
#         print(possible_sums)
#     return (sum(nums) / 2.)  in possible_sums 
canPartition([ 4,5,1, 10,7,1,4])

6 [4, 5, 1, 7, 1, 4]
i 0 4
2 [5, 1, 7, 1, 4] False
-3 [1, 7, 1, 4] False
i 1 5
1 [1, 7, 1, 4] True


True

In [2]:
# 419. Battleships in a Board,二维数组里横和直的个数
def countBattleships(board):
    if len(board) == 0: return 0
    m, n = len(board), len(board[0])
    count = 0
    for i in range(m):
        for j in range(n):
            if board[i][j] == 'X' and (i == 0 or board[i-1][j] == '.') and (j == 0 or board[i][j-1] == '.'):
                count += 1
    return count
countBattleships(
[["X",".",".","X"],[".",".",".","X"],[".",".",".","X"]])

In [67]:
错的
# def addOperators(num, target):
#     """
#     :type num: str
#     :type target: int
#     :rtype: List[str]
#     """
#     ans = []

#     def operator(cur,index,path,temp,flag,num,target,ans):
#         if index ==len(num):
#             if sum(cur) == target:
#                 print(cur,path,target)
#                 ans.append(path)
#             return
#         elif cur==[]:
#             operator([int(num[index])],index+1,path+num[index],temp,flag,num,target,ans)
#         else:
#             print(cur)
#             if cur[-1]!=0 and flag2:
#                 operator(cur[:-1]+[10*cur[-1]+int(num[index])],index+1,path+num[index],temp,flag,num,target,ans)
#             operator(cur[:-1]+[cur[-1]*int(num[index])],index+1,path+'*'+num[index],num[index],0,num,target,ans)
# #             if flag:
# #                 operator(cur[:-1]+[cur[-1]//int(temp)*(int(temp)*10+int(num[index]))],index+1,path+num[index],str((int(temp)*10+int(num[index]))),1,num,target,ans)
# #                 operator(cur[:-1]+[cur[-1]//int(temp)*(int(temp)*10+int(num[index]))],index+1,path+num[index],str((int(temp)*10+int(num[index]))),0,num,target,ans)
#             operator(cur+[int(num[index])],index+1,path+'+'+num[index],temp,flag,num,target,ans)
#             operator(cur+[-int(num[index])],index+1,path+'-'+num[index],temp,flag,num,target,ans)
#         return

#     operator([],0,"","",0,num,target,ans)
    
#     return ans

In [26]:
# 473. Matchsticks to Square,是否可以平分为4等份


def makesquare(nums):
    """
    :type nums: List[int]
    :rtype: bool
    """
    if not nums:
        return False
    sums = sum(nums)
    if sums%4!=0:
        return False
    nums.sort(reverse=True)
    target = sums//4
    def dfs(idx,cursum):
        if idx==len(nums) and cursum == [target,target,target,target]:
            return True
        if idx==len(nums):
            return False
        temp = False
        for i in range(4):
            if cursum[i]+nums[idx]<=target:
                cursum[i]+=nums[idx]
                temp = temp | dfs(idx+1,cursum)
                cursum[i]-=nums[idx]
            if temp:
                return True
        return temp

    return dfs(0,[0,0,0,0])

makesquare([1,1,2,2,2])


# 698. Partition to K Equal Sum Subsets
# The key is, cursum[i] == 0 means for all k > i, cursum[k] == 0
# because this algorithm always fill the previous buckets before trying the next.
def canPartitionKSubsets(nums, k):
    """
    :type nums: List[int]
    :type k: int
    :rtype: bool
    """
    if not nums:
        return False
    sums = sum(nums)
    if sums%k!=0:
        return False
    nums.sort(reverse=True)
    target = sums//k
    def dfs(idx,cursum):
#         print(cursum)
        if idx==len(nums) and cursum == [target for _ in range(k)]:
            return True
        if idx==len(nums):
            return False
        temp = False
        for i in range(k):
            if cursum[i]+nums[idx]<=target:
                cursum[i]+=nums[idx]
                temp = temp | dfs(idx+1,cursum)
                cursum[i]-=nums[idx]
            if temp:
                return True
            if cursum[i]==0:#这一步很重要，如果没有，会慢很多
                break
        return temp

    return dfs(0,[0 for _ in range(k)])

canPartitionKSubsets(nums = [4, 3, 2, 3, 5, 2, 1], k = 4)

True

In [2]:
# 1273. Delete Tree Nodes,删除和为0的子树
import collections
def deleteTreeNodes(nodes, parent, value):
    """
    :type nodes: int
    :type parent: List[int]
    :type value: List[int]
    :rtype: int
    """
    parents = collections.defaultdict(list)#每个parents拥有的child
    for i,node in enumerate(parent):
        parents[node].append(i)

    def calSum(node):#这个节点及子节点的和,这个节点及其以下还幸存的节点个数
#         if len(parents[node])==0:
#             if value[node]==0:
#                 return (0,0)
#             return (value[node],1)
        cursum,curnodes = value[node],1
        for i in parents[node]:
            r1,r2 = calSum(i)
            cursum+=r1
            curnodes+=r2
        return (cursum,curnodes) if cursum!=0 else (0,0)
    return calSum(0)[1]
deleteTreeNodes(nodes = 7, parent = [-1,0,0,1,2,2,2], value = [1,-2,4,0,-2,-1,-1])

2

In [2]:
# 851. Loud and Rich，给定树结构，求每个的比它小的节点
# 从叶子出发，一层层推进，中途更新值
def loudAndRich(richer, quiet):
    """
    :type richer: List[List[int]]
    :type quiet: List[int]
    :rtype: List[int]
    """
    # [1,2],1>2
    large = {i:set() for i in range(len(quiet))}#每个元素与比他大的{2:1}
    small = {i:set() for i in range(len(quiet))}#每个元素和比他小的{1:2}
    for i,j in richer:
        large[j].add(i)
        small[i].add(j)

    res = [i for i in range(len(quiet))]
    visited = set()

    while len(visited)<len(res):
        nodes = [i for i in large if len(large[i])==0 and (i not in visited)]
        for node in nodes:
            visited.add(node)
            for key in list(small[node]):
                if quiet[res[node]]<quiet[res[key]]:
                    res[key] = res[node]
                large[key].remove(node)
    return res
loudAndRich(richer = [[1,0],[2,1],[3,1],[3,7],[4,3],[5,3],[6,3]], quiet = [3,2,5,4,6,1,7,0])

[5, 5, 2, 5, 4, 5, 6, 7]

In [1]:
# 756. Pyramid Transition Matrix
# 为什么这里用的DFS而不是用BFS,BFS需要每次把每一轮的所有选项全部跑完
# 但这里DFS第一轮跑完后，直接对每一个状态走下一轮，直到可能走完，一旦有正确的，就可以了
import itertools
def pyramidTransition( bottom, allowed):
    """
    :type bottom: str
    :type allowed: List[str]
    :rtype: bool
    """
    allow = dict()
    for node in allowed:
        if node[:2] not in allow:
            allow[node[:2]] = []
        allow[node[:2]].append(node[-1])

    def dfs(status):
        if len(status)==1:
            return True
        nxts = []
        for j in range(len(status)-1):
            if status[j]+status[j+1] in allow:
                nxts.append(allow[status[j]+status[j+1]])
            else:#找不到下一个，必须返回False，否则会出现错误
                return False
        for key in itertools.product(*nxts):#星号可以在一个列表内对元素进行映射
            if dfs(key):
                return True
        return False

    return dfs(bottom)
pyramidTransition(bottom = "BCD", allowed = ["BCG", "CDE", "GEA", "FFF"])

True